# PySpark SQL Query Example

The following Spark Job uses PySpark SQL engine to query Reporters from a Parquet file.


In [1]:
import os
import sys
import time
import datetime

from os.path import expanduser

import findspark
findspark.init()

from pyspark import SparkConf, SparkContext

from pyspark.sql import SparkSession, SQLContext
from pyspark.sql import functions as f

In [2]:
# Simple function to get file size
def getSize(filename):
    st = os.stat(filename)
    return st.st_size

# In-File Location ( normally an Azure Data Lake or AWS s3://<file-location-bucket> )
home_dir = expanduser("~")
parquet_file = os.path.join(home_dir, 'Dev/Data/wspr/wsprspots-2020-11.snappy')
file_size = getSize(parquet_file)

# Setup the Spark Cluster Config Variables
conf = SparkConf().setAppName("Radio Data Science Parquet Read").setMaster("local[*]")

# Instantiate the Spark Session
spark = SparkSession \
    .builder \
    .appName("Radio Data Science - Parquet Read Example") \
    .config(conf=conf) \
    .getOrCreate()

# Print some basic header information
print(f'\n* Reading file ..: {os.path.basename(parquet_file)}')
print(f'* Timestamp .....: {datetime.datetime.now()}')
print(f'* File Size .....: {file_size:,} bytes compressed')

# Read the Parquet file
start = time.time()
df = spark.read.load(parquet_file, format="parquet")
end = time.time()
print(f"* Read Time .....: {round((end-start), 3)} sec")

# This is a second trip through the file to get a total count
start = time.time()
print(f'* Record Count ..: {df.count():,}')
end = time.time()
print(f"* Count Time ....: {round((end-start), 3)} sec")

#
# Example-1: Group by aggregation using DataFrames
#
print(f'\nGroup, Count, Order by using DataFrame')
start = time.time()
df2 = df.groupBy('Reporter').count().orderBy('count', ascending=False)
df2.show(5)
end = time.time()
print(f'Query Time ....: {round((end-start), 3)} sec\n')

#
# Example-2: Group by aggregation using PySparkSQL ( sql query language )
#
print(f'\nGroup, Count, Order by using PySpark SQL Query')
start = time.time()
df.createOrReplaceTempView("spot_data")
groupDF = spark.sql("SELECT Reporter, COUNT(*) FROM spot_data GROUP BY Reporter ORDER BY count(1) DESC")
groupDF.show(5)
end = time.time()
print(f"Query Time ....: {round((end-start), 3)} sec\n")


* Reading file ..: wsprspots-2020-11.snappy
* Timestamp .....: 2020-12-31 05:50:15.525111
* File Size .....: 1,151,378,369 bytes compressed
* Read Time .....: 1.485 sec
* Record Count ..: 77,337,023
* Count Time ....: 0.899 sec

Group, Count, Order by using DataFrame
+--------+-------+
|Reporter|  count|
+--------+-------+
|  EA8BFK|1120739|
|  OE9GHV|1103335|
|   WA2TP| 847124|
|   KD2OM| 834896|
|  IW2NKE| 818315|
+--------+-------+
only showing top 5 rows

Query Time ....: 5.966 sec


Group, Count, Order by using PySpark SQL Query
+--------+--------+
|Reporter|count(1)|
+--------+--------+
|  EA8BFK| 1120739|
|  OE9GHV| 1103335|
|   WA2TP|  847124|
|   KD2OM|  834896|
|  IW2NKE|  818315|
+--------+--------+
only showing top 5 rows

Query Time ....: 5.368 sec



In [3]:
# Shutdown the PySpark engine.
spark.stop()